In [36]:
import os
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import matplotlib.pylab as plt
import time

%matplotlib inline

from urllib.request import urlopen
from pathlib import Path
from collections import defaultdict
from random import choice, sample, shuffle
from numpy.random import choice

In [102]:
f = open('./Material_de_interes/tc02Data/yeast_LIT_Reguly.txt',"r")
r = open('./Material_de_interes/tc02Data/yeast_LIT_RegulyBIS.txt',"w")
lines=f.readlines()
lista_red = []
for linea in lines:
    lista_red.append(linea.split('\t')[:2])
    
for enlace in lista_red[1:]:
    print('%s' %' '.join(enlace), file=r)
f.close()
r.close()
red_lit2 = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_LIT_RegulyBIS.txt')

In [103]:
red_lit = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_LIT.txt')
red_apms = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_AP-MS.txt')
red_bin = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_Y2H.txt')
red_lit2 = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_LIT_RegulyBIS.txt')

In [38]:
esenciales = open('./Datos/Essential_ORFs_paperHe2.txt')
lista_esenciales = []

for line in esenciales:
    lista_esenciales.append(line.rstrip('\n'))

In [39]:
def nodos_esenciales(red, lista_atr = lista_esenciales, atributo = 'Esencial'):
    
    dict_vacio = dict()
    nx.set_node_attributes(red, dict_vacio.fromkeys(list(red.nodes()),0), atributo )
    
    for nodo in lista_atr:
        try:
            red.nodes[nodo][atributo] = 1
        except KeyError:
            continue
            
    return None

In [104]:
nodos_esenciales(red_lit)
nodos_esenciales(red_bin)
nodos_esenciales(red_apms)
nodos_esenciales(red_lit2)

In [41]:
def nodosEsenciales(red, esenciales = lista_esenciales, atributo = 'Esencial'):
    red_aux = red.copy()
    red_aux.remove_nodes_from(list(nodosNoEsenciales(red).nodes()))
    return red_aux.nodes()

In [42]:
def nodosNoEsenciales(red, esenciales = lista_esenciales):
    red_aux = red.copy()
    red_aux.remove_nodes_from(esenciales)
    return red_aux

In [43]:
def sacarEsenciales(red):
    red_aux = red.copy()
    GC = max(nx.connected_component_subgraphs(red_aux), key=len)
    n = GC.number_of_nodes()
    red_aux.remove_nodes_from(list(nodosEsenciales(red)))
    GC = max(nx.connected_component_subgraphs(red_aux), key=len)
    m = GC.number_of_nodes()
    return m/n

In [44]:
sacarEsenciales(red_lit)

0.281121187139324

In [45]:
def sampleNodos(red,dic1,dic2,k,q,dif1=0):
    if dif1 == 0:
        dif1 = dic1[k][1]
    try:
        dif2 = dic1[q][0] - dif1
        sample_nodos = sample(dic2[q],dif1)
        red.remove_nodes_from(sample_nodos)
        if dif2 <= 0:
            return None
        else:
            sampleNodos(red,dic1,dic2,k,q-1,dif1=dif2)
            
    except:
        sampleNodos(red,dic1,dic2,k,q-1,dif1=0)

In [81]:
def remover_nodos_no_esenciales(red, indice, lista_grados, tot_remover, lista_no_esenciales_grado,
                                prohibidos, removidos = 0):
    
    #prohibidos es una lista vacia en el primer caso
    
    grado = lista_grados[indice]
    # miro los nodos no esenciales con ese grado
    #nodos_grado = lista_no_esenciales_grado[grado]
    n = len(lista_grados)
    i = 0
    while removidos < tot_remover:
        m = len(lista_no_esenciales_grado[grado])
        if m > 0:
            
            nodos_a_remover = np.random.choice(lista_no_esenciales_grado[grado], min(m, tot_remover - removidos),
                                               replace = False)
            # Filtro por las dudas
            nodos = [nodo for nodo in nodos_a_remover if nodo not in prohibidos]
            red.remove_nodes_from(nodos)
            for nodo in nodos:
                lista_no_esenciales_grado[grado].remove(nodo)
            if len(nodos) == 0:
                i += 1
                lista_no_esenciales_grado[grado] = lista_no_esenciales_grado[lista_grados[min(indice+i,
                                                                                          n-1)]] + lista_no_esenciales_grado[lista_grados[max(indice-i, 0)]]
            removidos += len(nodos)
            # Agrego los prohibidos a la lista de nodos
            prohibidos = prohibidos + nodos
            
            #print('saco {}'.format(min(m, tot_remover, len(nodos))))
            #print(lista_no_esenciales_grado[grado])
            
        else:
            i += 1
            lista_no_esenciales_grado[grado] = lista_no_esenciales_grado[lista_grados[min(indice+i, n-1)]] + lista_no_esenciales_grado[lista_grados[max(indice-i, 0)]]
            #nodos = np.choice(lista_no_esenciales_grado[grado], min(m, tot_remover), replace = False)
            #red.remove_node(nodo)
            #lista_no_esenciales_grado[grado].remove(nodos)

    # para un chequeo

    # print('prohibi: {}'.format(len(prohibidos)))
        
    return red, lista_no_esenciales_grado, prohibidos

In [82]:
def randomNoEsenciales(red, GC):
    red_aux = red.copy()
    n = red.number_of_nodes()
    
    #GC = max(nx.connected_component_subgraphs(red), key = len)
    n_gc = GC.number_of_nodes()
    
    degs = sorted(dict(red.degree()).values(), reverse=True)
    nodos = dict(red.nodes(data=True))
    grados = {d: [0,0] for d in degs}
    noEsencXGrado = defaultdict(list)
    for nodo, deg in red.degree():
        grados[deg][0] += nodos[nodo]['Esencial']
        grados[deg][1] += 1-nodos[nodo]['Esencial']
        if (not nodos[nodo]['Esencial']):
            #print(nodos)
            noEsencXGrado[deg].append(nodo)
    grad = sorted(list(grados.keys()))
    
    # almacenamos las cantidades de nodos en el dataframe para que sea mas facil manipular
    df_nodos = pd.DataFrame(grados).T
    df_nodos.columns = ['esenciales', 'no esenciales']
    df_nodos['diferencia'] = df_nodos['no esenciales'] - df_nodos['esenciales']   
    
    #list(df[df.esenciales>=1].index)
    #print(grad)
    #print(noEsencXGrado)
    
    diferencias = df_nodos.diferencia
    
    #print(df_nodos)
    prohibidos = []
    for grado, ind in zip(grad, range(len(grad))):
        # Para ese grado, saco tantos nodos no esenciales como nodos esenciales hay con ese grado
        # Cantidad de nodos a remover para un grado fijo
        tot_es = df_nodos.loc[grado].esenciales
        
        #print("Total a sacar {}".format(tot_es))
        #print("Grado: {}, Indice: {}".format(grad[ind],ind))
        red_aux, noEsencXGrado, prohibidos = remover_nodos_no_esenciales(red_aux, ind, grad, tot_es,
                                                                          noEsencXGrado, prohibidos)
        #print("removidos {}".format(removidos))
        
    GC_posterior = max(nx.connected_component_subgraphs(red_aux), key = len)
    m = GC_posterior.number_of_nodes()
    
    return m/n_gc, grados, grad, df_nodos

In [85]:
gc = max(nx.connected_component_subgraphs(red_apms), key = len)
k, gr, acum, df = randomNoEsenciales(red_apms, gc)

In [72]:
errores_apms = []
gc = max(nx.connected_component_subgraphs(red_apms), key = len)
start = time.time()
for i in range(1000):
    
    gc_size, grados, gr, df = randomNoEsenciales(red_apms, gc)
    errores_apms.append(gc_size)
    
print(time.time()-start)

352.4165229797363


In [86]:
a = (np.mean(errores_apms), np.std(errores_apms), sacarEsenciales(red_apms))

In [74]:
errores_lit = []
gc = max(nx.connected_component_subgraphs(red_lit), key = len)
start = time.time()
for i in range(1000):
    
    gc_size, grados, gr, df = randomNoEsenciales(red_lit, gc)
    errores_lit.append(gc_size)
    
print(time.time()-start)

In [87]:
b = (np.mean(errores_lit), np.std(errores_lit), sacarEsenciales(red_lit))

In [76]:
errores_bin = []
gc = max(nx.connected_component_subgraphs(red_bin), key = len)
start = time.time()
for i in range(1000):
    gc_size, grados, gr, df = randomNoEsenciales(red_bin, gc)
    errores_bin.append(gc_size)
    
print(time.time()-start)

In [88]:
c = (np.mean(errores_bin), np.std(errores_bin), sacarEsenciales(red_bin))

In [105]:
errores_lit2 = []
gc = max(nx.connected_component_subgraphs(red_lit2), key = len)
start = time.time()
for i in range(1000):
    
    gc_size, grados, gr, df = randomNoEsenciales(red_lit2, gc)
    errores_lit2.append(gc_size)
    
print(time.time()-start)

328.57858633995056


In [106]:
d = (np.mean(errores_lit2), np.std(errores_lit2), sacarEsenciales(red_lit2))

In [107]:
# Tabla
tabla3 = pd.DataFrame([a,b,c,d])
tabla3.columns = ['Random no esenciales', 'Desvio', 'Esenciales']
tabla3.index = ['APMS','Literatura','Y2H', 'Reguly']

In [108]:
tabla3

,Random no esenciales,Desvio,Esenciales
APMS,0.379465,0.019438,0.323705
Literatura,0.417350,0.004811,0.281121
Y2H,0.622151,0.011863,0.624165
Reguly,0.540567,0.004132,0.575062
